# Project Euler problem 192 - Best Approximations

[Link to problem on Project Euler homepage](https://projecteuler.net/problem=192)

## Description

Let $x$ be a real number.
A *best approximation* to $x$ for the denominator bound $d$ is a rational number $r/s$ in reduced form, with $s \leq d$, such that any rational number which is closer to $x$ than $r/s$ has a denominator larger than $d$:

$$|p/q-x| < |r/s-x| \Rightarrow q > d$$

For example, the best approximation to $\sqrt{13}$ for the denominator bound $20$ is $18/5$ and the best approximation to $\sqrt{13}$ for the denominator bound $30$ is $101/28$.

Find the sum of all denominators of the best approximations to $\sqrt{n}$ for the denominator bound $10^{12}$, where $n$ is not a perfect square and $1 < n \leq 100000$.

In [5]:
from math import sqrt
from itertools import cycle

def continued_fraction_periodic(d):

    sd = sqrt(d)
    p, q = 0, 1

    terms = []
    pq = {}

    while (p, q) not in pq:
        pq[(p, q)] = len(terms)
        terms.append(int((p + sd)/q))
        # black magic to determine period
        p = terms[-1]*q - p
        q = (d - p**2)//q
        yield terms[-1]

    i = pq[(p, q)]
    
    for term in cycle(terms[i:]):
        yield term

In [6]:
def best_approximation_convergents(n, bound):
    cf_iter = continued_fraction_periodic(n) # continued fraction iterator
    
    s0 = next(cf_iter)
    s1 = next(cf_iter)
    
    # initialize rational approximations
    p = [None, s0, s0*s1 + 1] # numerator
    q = [None,  1, s1] # denominator
    
    for a in cf_iter:
        p[0], q[0] = p[1], q[1]
        p[1], q[1] = p[2], q[2]
        p[2], q[2] = p[0] + a*p[1], q[0] + a*q[1]

        if q[2] > bound:
            break
    return p[:2], q[:2], a

print(best_approximation_convergents(4850, 10**12))

([17721201717166, 22555308267375], [254461627067, 323875351814], 4)


In [7]:
def best_approximation_semiconvergents(n, bound):
    
    p, q, a = best_approximation_convergents(n, bound)
    
    p_best, q_best = p[1], q[1] # current estimate of best approximation
    
    if a == 1: # no semi convergents. Convergent is best approximation
        return p_best, q_best

    if a % 2 == 0: # if a is even we have to check if first semi convergent better approximation
        p_sc, q_sc = p[0] + a*p[1]//2, q[0] + a*q[1]//2
        
        if q_sc > bound: # if semi convergent exceeds bound return best convergent approximation
            return p_best, q_best

        if q_best**2*abs(p_sc**2 - q_sc**2*n) < q_sc**2*abs(p_best**2 - q_best**2*n):
            p_best, q_best = p_sc, q_sc
    
    alim = a//2 + 1
    
    for _a in range(a//2+1, a):
        p_sc, q_sc = p[0] + _a*p[1], q[0] + _a*q[1]
        
        if q_sc > bound:
            break
        
        p_best, q_best = p_sc, q_sc
    
    return p_best, q_best

print(best_approximation_semiconvergents(4850, 10**12))
print(best_approximation_semiconvergents(13, 30))

(62831818251916, 902212330695)
(101, 28)


In [9]:
from time import time
from math import sqrt

t0 = time()

limit = 10**5
bound = 10**12

squares = [i**2 for i in range(2, int(sqrt(limit))+1)]

total = 0
for n in range(2, limit+1):
    if n in squares:
        continue
    
    p, q = best_approximation_semiconvergents(n, bound)
    total += q

seconds = time() - t0

minutes, seconds = divmod(seconds, 60)
hours, minutes = divmod(minutes, 60)

print("Result: {}".format(total))
print("Time elapsed = {:.0f}h {:.0f}m {:f}s".format(hours, minutes, seconds))

Result: 57060635927998347
Time elapsed = 0h 0m 3.540848s
